<a href="https://colab.research.google.com/github/1306minyoung/data/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**정형, 비정형 데이터**

정형: csv 파일처럼 구조를 가지고 있는 데이터
비정형: 텍스트, 사진, 디지털 음악 등 구조를 가지고 있지 않은 데이터

**앙상블 학습**

정형데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘

대부부 결정 트리 기반임


**랜덤 포레스트**

앙상블 학습 중 가장 유명하고 안정적인 성능 제공

여러 개의 결정트리를 랜덤하게 만들고 각 결정 트리의 예측을 사용해 최종 예측 출력

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
#oob_score : 부트스트랩 샘플에 포함되지 않는 샘플
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


##엑스트라 트리

랜덤 포레스트와 비슷하게 동작하나,
부트 스트랩 샘플 사용 안하고, 각 결정트리를 만들 때 전체훈련 세트를 사용

전체 특성중에 일부 특성을 래덤하게 선택해 노드를 분할하는 데 사용한다.

In [6]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


##그레디언트 부스팅

깊이가 얕은 결정트리를 사용해 이전 트리의 오차를 보완하는 방식.
과대적합에 강하고, 일반적으로 높은 일반화 성능을 기대할 수 있음.

경사각을 쪼끔씩 바꿔가면서 점 위치가 바뀜....(?)

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


##히스토그램 기반 부스팅

정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기가 높은 알고리즘

입력 특성을 256개의 구간으로 나누므로 노드를 분할할때 최적의 분할을 매우 빠르게 찾을 수 있음

256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함

즉. 어떤 샘플에 누락된 값이 있다면 떼어놓은 구간의 값을 대신 사용합 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음

히스토그램 : 데이터를 일정한 구간으로 쪼개서 막대그래프로 표현

입력 데이터의 각 특성을 256개의 구간으로 나누어 256개의 값으로 변환

예를 들어, 입력 데이터는 하나의 특성만을 가지고 있고, 특성에 1부터 512까지의 수가 분포되어 있다고 하자 그렇다면 1,2를 1로, 3,4를 2로 치환하는 식으로 특성의 범위를 256개의 구간으로 축소할 수 있음

따라서 최적의 분할을 찾기 위해서 각 특성단 256번만 계산을 하면 되므로 효율적임

특히 실수 범위를 정수로 범위를 축소할 수 있다는 점이 뛰어남

max_iter: 부스팅 반복 횟수 (n_estimators 대신 사용함).

성능을 높이려면 이 매개변수를 조절

In [11]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [12]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

##XGBoost

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


##LightGBM

In [16]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
